<a href="https://colab.research.google.com/github/tomonari-masada/course2023-stats2/blob/main/03_NumPyro_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NumPyro入門 (2)

## 準備

In [ ]:
!pip install arviz
!pip install numpyro

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import jax
from jax import random
import numpyro
import numpyro.distributions as dist
from numpyro.infer import NUTS, MCMC, Predictive

import arviz as az

%config InlineBackend.figure_format = 'retina'

rng_key = random.PRNGKey(0)
az.style.use("arviz-darkgrid")
numpyro.set_platform("cpu")

## NumPyroによるMCMC

### MCMC（マルコフ連鎖モンテカルロ）とは
* ベイズ的なモデリングにおいて、事後分布を知ろうとする方法の一つ。
 * もう一つの方法に、変分推論(variational inference)がある。
* MCMCは、事後分布からのサンプルを通して、事後分布そのものを知ろうとする方法。
* モデルが複雑になるほど、事後分布$p(\theta|X)$からのサンプリングは、難しくなっていく。
 * 比較的シンプルなモデルについては、うまいサンプリング手法を構成できる（例：LDAのcollapsed Gibbs sampling）。
* この授業では、MCMCの実際上の使い方を説明する。理屈はあまり説明しない。

### NumPyroにおける確率モデルの定義

* NumPyroでは、ベイズ的モデルを関数として定義する。

**例題**
 * 数値データがたくさんある。標準偏差は1らしい。平均は0に近いが、0からずれているかもしれない。この平均を知りたい。
 * ベイズ的なモデリングによって、平均がいくらの可能性がどのくらいあるかを表す、事後分布を得ることにする。

* そこで、下記のモデルを使う（前回すでに使っていたモデル）。
$$ \mu \sim N(0, 0.5) $$
$$ x \sim N(\mu, 1) $$
 * 事後分布は$p(\mu|X)=\frac{p(X|\mu)p(\mu)}{p(X)}=\frac{p(\mu)\prod_{i=1}^N p(x_i|\mu)}{p(X)}$

* 観測データは乱数で準備する。

In [ ]:
rng_key, rng_key_ = random.split(rng_key)
observed = jax.random.normal(rng_key_, (100,))

* 上述のモデルをNumPyroで書くと、以下のようになる。

In [ ]:
def model(data=None):
  mu = numpyro.sample("mu", dist.Normal(0, 0.5))
  obs = numpyro.sample("obs", dist.Normal(mu, 1), obs=data)

### NumPyroによるMCMCの実行

* `mu`が従う事後分布$p(\mu|X)$からサンプルを得るには、以下のようにすればよい。
* `num_warmup`は、最初の何個のサンプルを捨てるかを指定する引数。
 * MCMCで得られるサンプルは、最初のほうのものは、通常、捨てる。

In [ ]:
kernel = NUTS(model)
mcmc = MCMC(kernel, num_warmup=1000, num_samples=2000, num_chains=4)
rng_key, rng_key_ = random.split(rng_key)
mcmc.run(rng_key_, data=observed)

* chainとは、サンプルの一つの系列のこと。
* MCMCによるサンプリングでは、直前のサンプルを少し変化させることで、次のサンプルを求める。
 * 前回説明したとおり、この変化のさせ方が賢いので、ちゃんと事後分布からのサンプルになる。
* ということは、サンプルは、初期値から始まって、一筋の系列をなしている。この系列をchainと呼ぶ。
* chainが一つだけだと心もとないので、普通は、複数のchainを走らせる。
 * 別々のchainで、サンプルの分布が大きく違っていたりすると、何かがおかしいと分かる。

* NUTS (No-U-Turn-Sampler) は、よく使われるサンプラー。
* HMCの改良版。詳細は割愛するが、おおよその説明は後ほど。
 * https://mc-stan.org/docs/reference-manual/hmc.html
 * https://arxiv.org/abs/1111.4246

* サンプルの統計量を見てみる。

In [ ]:
mcmc.print_summary()

**注意** divergenceが0でなかったら、色々と考え直す必要がある。（後の回で説明します。）
 

* サンプルを取得する。

In [ ]:
samples = mcmc.get_samples()
samples

In [ ]:
type(samples['mu'])

* Numpyのndarrayに変換

In [ ]:
type(np.array(samples['mu']))

* ArviZ向けのデータへ変換する。
 * こうすると、ArviZの様々な機能が使えるようになる。

In [ ]:
idata = az.from_numpyro(mcmc)
idata

* 上の実行例では、chainは4本あり、それぞれ2000のサンプルから成っている。

In [ ]:
idata.posterior["mu"].shape

* 特定のchainだけ選ぶ方法は以下の通り。

In [ ]:
idata.posterior["mu"].sel(chain=0).shape

* 特定のchainの、最初の10個のサンプルだけ見てみる。

In [ ]:
idata.posterior["mu"].sel(chain=0)[:10]

* chainの中身のデータ型はxarrayの配列。
 * NumPyの配列とは違う。

In [ ]:
type(idata.posterior["mu"])

* `.data`でndarrayへ変換できる。

In [ ]:
type(idata.posterior["mu"].data)

In [ ]:
idata.posterior["mu"].data.shape

* サンプルのヒストグラムを描くと、事後分布の大体の形が分かる。
* ここでは、あえて、arvizを使わずにヒストグラムを描いてみる。

In [ ]:
import seaborn as sns

df = pd.DataFrame(data=idata.posterior["mu"].data.T)
sns.displot(df, kind="kde", rug=True);

* とはいえ、やはり`arviz`を使う方が良い。

In [ ]:
az.plot_trace(idata);

* さて、chainが４本からなるこのサンプルは、うまく事後分布を表しているのだろうか？
 * 以下、サンプルの分析手法を紹介する。

## MCMCの結果の分析
 * https://github.com/pymc-devs/pymc-examples/blob/main/examples/diagnostics_and_criticism/Diagnosing_biased_Inference_with_Divergences.ipynb
 * https://www.statlect.com/fundamentals-of-statistics/Markov-Chain-Monte-Carlo-diagnostics

* MCMCを使うときには、得られたchainの良し悪しを気にしないといけない。
* 例えば、chainが事後分布の定義域のごく狭い範囲しか踏査していないかもしれない。


**例題**
* 観測データは正規分布$N(\mu, \sigma^2)$に従うと仮定。
* $\mu$と$\sigma$について事前分布を導入。
$$\begin{align}
\mu & \sim N(0, 10) \\
\sigma & \sim \text{HalfNormal}(1) \\
x & \sim N(\mu, \sigma^2)
\end{align}$$
 * [half-normal分布](https://en.wikipedia.org/wiki/Half-normal_distribution)は、平均0の正規分布に従う確率変数の絶対値が従う分布。
 

In [ ]:
def model(data=None):
  mu = numpyro.sample("mu", dist.Normal(0, 10))
  sd = numpyro.sample("sd", dist.HalfNormal(1))
  obs = numpyro.sample("obs", dist.Normal(mu, sd), obs=data)

In [ ]:
kernel = NUTS(model)
mcmc = MCMC(kernel, num_warmup=10, num_samples=1000, num_chains=4)
rng_key, rng_key_ = random.split(rng_key)
mcmc.run(rng_key_, data=observed)

* arvizでサンプルを可視化する。
 * デフォルトではkernel density estimates。ヒストグラムも選べる。
 * 全サンプルのプロットも右半分に描かれる。

In [ ]:
idata = az.from_numpyro(mcmc)
idata

In [ ]:
az.plot_trace(idata);

* pair plotで2変数の分布を同時に見る。
 * 周辺分布も表示させることができる。

$$\begin{align}
p(\mu, \sigma | X) & \propto p(X | \mu, \sigma) p(\mu) p(\sigma) & {（事後分布）} \\
p(\mu | X) & = \int p(\mu, \sigma | X) d\sigma & \mbox{（$\mu$の周辺事後分布）} \\
p(\sigma | X) & = \int p(\mu, \sigma | X) d\mu & \mbox{（$\sigma$の周辺事後分布）}
\end{align}$$

In [ ]:
az.plot_pair(idata, marginals=True, divergences=True);

### 自己相関
* 自己相関は小さいほど良い。
 * 小さいほど、サンプルが相互に独立だとみなせる。
* 比較的大きなラグ(lag)でも相関が0に近くない場合は、問題あり。
* 参考資料
 * https://www.statlect.com/fundamentals-of-statistics/autocorrelation


In [ ]:
az.plot_autocorr(idata, var_names=["mu", "sd"]);

* 横軸がラグ。

### Gelman-Rubin統計量
* 定量的にサンプルの良し悪しを分析できる。
* R-hatと呼ばれる値が1に近いほど、連鎖の分布がより収束している。
 * https://mc-stan.org/docs/reference-manual/analysis.html を参照。
 * http://www.omori.e.u-tokyo.ac.jp/MCMC/mcmc.pdf の6.2.2を参照。
* R-hatは1.05より小さいことが望ましいらしい。
 * https://www.youtube.com/watch?v=WbNmcvxRwow

In [ ]:
az.summary(idata)

* mcse, essについては、下記を参照。
 * https://mc-stan.org/docs/2_26/reference-manual/effective-sample-size-section.html

 ### 3.3.4 HDI (highest density interval)
* HDIが何であるかについては下記を参照。
 * http://web.sfc.keio.ac.jp/~maunz/BS14/BS14-11.pdf
 * https://www.sciencedirect.com/topics/mathematics/highest-density-interval 
* arvizのforest plot
  * デフォルトではHDI=94.0%の区間を図示する。
  * r_hat=TrueでR-hat統計量も図示する。

In [ ]:
az.plot_forest(idata, r_hat=True);

* ridge plot

In [ ]:
axes = az.plot_forest(idata,
                      kind='ridgeplot',
                      ridgeplot_truncate=False,
                      ridgeplot_quantiles=[.25, .5, .75],
                      ridgeplot_alpha=.7,
                      colors='white',
                      figsize=(9, 7))
axes[0].set_title('Estimated mu and sd');

* 似ているが別の可視化。
 * https://sites.google.com/site/doingbayesiandataanalysis/ この本の流儀による可視化だそうです（がよく知りません・・・）。
 * これが分かりやすいかもしれません。

In [ ]:
az.plot_posterior(idata);



---



---



## Hamiltonian Monte Carlo (HMC)
* ここでは直感的な説明をするにとどめる。
* Stanのマニュアルを参考にした。
 * https://mc-stan.org/docs/reference-manual/hamiltonian-monte-carlo.html
* その他の参考資料
 * https://ryokamoi.github.io/blog/tech/2018/12/09/hmc

### 補助変数
* 密度関数$p(\theta)$からのサンプリングを実現したいとする。
 * $p(\theta)$については、規格化定数は不明でも構わない。
* HMCでは、補助変数$\rho$を追加し、同時分布$p(\rho, \theta) = p(\rho|\theta)p(\theta)$からのサンプリングをおこなう。
* 多くの場合（Stanでも）、$\rho$の値が従う分布は、$\theta$に依存しない多変量正規分布だと仮定する。
$$\rho \sim \text{MultiNormal}(0, M)$$
 * $M$は対角成分しか持たないらしい（Stanのマニュアル参照）。

### leapfrogアルゴリズム
* $V(\theta) \equiv - \ln p(\theta)$および$H(\rho, \theta) = - \ln p(\rho, \theta)$と定義する。
* leapfrogアルゴリズムでは、以下のように$\theta$を更新することで、サンプルのchainを作る。

1. $\rho$を$\text{MultiNormal}(0,M)$からdraw
2. 以下の一連の更新式を$L$回繰り返し実行する。
$$\begin{align}
\rho & \leftarrow \rho - \frac{\epsilon}{2}\frac{\partial V}{\partial \theta}
\notag \\
\theta & \leftarrow \theta + \epsilon M^{-1}\rho
\notag \\
\rho & \leftarrow \rho - \frac{\epsilon}{2}\frac{\partial V}{\partial \theta}
\end{align}$$
 * この結果、$\rho$は$\rho^*$へ、$\theta$は$\theta^*$へ、それぞれ更新されたとする。
3. この$\rho^*, \theta^*$を、確率$\min(1, \exp(H(\rho, \theta) - H(\rho^*, \theta^*)))$で、次のサンプルとして採用する。
 * 採用されなければ、元の$\rho,\theta$をそのまま次でも使う。



### the no-U-turn sampling (NUTS) アルゴリズム
* leapfrogアルゴリズムで、$M$と$\epsilon$と$L$は、適切に調整すべきパラメータである。
* これらのパラメータを自動的に調整するアルゴリズムとしてHoffmanとGelmanにより提案されたのが、no-U-turn sampling (NUTS)。（終）

### divergence
* leapfrogアルゴリズムは、$\frac{\partial V}{\partial \theta}$を使っている。
* 雰囲気を言うと、これは、本当なら$p(\theta)$の地形に沿って滑らかに動きたいところを、一階の微分を使って近似的に動いている。
* 近似であるため、本当ならそう動きたいという軌道から、外れてしまうこともある。
* この外れ方が非常に大きくなってしまうことを、divergenceと呼ぶ。
* divergenceが大きいと、得られたサンプルchainが所望の密度関数からのサンプルchainとみなせない。

# 課題3
* HDIとは何かをしらべて、レポートしてください。